In [196]:
import numpy as np
import os 
import torch
import torch
import pandas as pd
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import Dataset
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import json

In [197]:
xdim = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'H': 8, 'I': 9, 'J': 10, 'K': 11}
ydim = [i for i in range(19, 0, -1)]
# print(xdim)
# print(ydim)
def get_hold_coords(hold, xdim = xdim, ydim = ydim):
    col = xdim[hold[0]] -1
    row = ydim[int(hold[1:])] - 1
    return (row, col)

In [198]:
moonboard_2016_filename = "../data/mb_problem_holds.csv"
moonboard_2016_size = (18,11)
df = pd.read_csv(moonboard_2016_filename)
print(len(df))
climb = df.iloc[10]
print(climb)
print(climb['Moves'])


542
Method                                                      Feet follow hands
Name                                                         ONE ARMED BANDIT
Grade                                                                      8A
UserGrade                                                                  8A
MoonBoardConfiguration                                                    NaN
MoonBoardConfigurationId                                                    0
Setter                      {'Id': '89b67a5a-62d2-46df-95d6-4a5e25b0c361',...
FirstAscender                                                           False
Rating                                                                      0
UserRating                                                                  5
Repeats                                                                     0
Attempts                                                                    0
Holdsetup                   {'Id': 1, 'Description': 'MoonBo

In [199]:
# #Split Data
# def split_csv_and_save(input_file, output_dir, train_size=0.8, random_state=None):
#     # Create the output directory if it doesn't exist
#     if not os.path.exists(output_dir):
#         os.makedirs(output_dir)
    
#     # Read the CSV file
#     data = pd.read_csv(input_file)
    
#     # Split the data into train and test sets
#     train_data, test_data = train_test_split(data, train_size=train_size, random_state=random_state)
    
#     # Define the paths for train and test files
#     train_file = os.path.join(output_dir, "train.csv")
#     test_file = os.path.join(output_dir, "test.csv")
    
#     # Save train and test data to CSV files
#     train_data.to_csv(train_file, index=False)
#     test_data.to_csv(test_file, index=False)

# # Example usage
# moonboard_2016_filename = "../data/mb_problem_holds.csv"
# output_directory = "../modeldata/splits"
# split_csv_and_save(moonboard_2016_filename, output_directory, train_size=0.8, random_state=42)

In [261]:
french_grade_order = ['4', '5', '5+', '6A', '6A+', '6B', '6B+', '6C', '6C+', '7A', '7A+',
                       '7B', '7B+', '7C', '7C+', '8A', '8A+', '8B', '8B+']
grade_bounds = ('6B+', '8A')
scale_min = french_grade_order.index(grade_bounds[0])
scale_max = french_grade_order.index(grade_bounds[1])
scale_bounds = (scale_min, scale_max)
scale_size = scale_max - scale_min

class ClimbDataset(Dataset):
    def __init__(self, climbs_filename, board_size, scale_bounds):
        self.climb_df = pd.read_csv(climbs_filename)
        self.board_size = board_size
        self.scale_min = scale_bounds[0]
        self.scale_max = scale_bounds[1]
        self.scale_size = self.scale_max - self.scale_min + 1
        
    def __len__(self):
        return len(self.climb_df)
    
    def __getitem__(self, idx):
        climb = self.climb_df.iloc[idx]
        grade = climb['Grade']
        moves = climb['Moves']
        moves = moves.replace("'", '"')
        moves = moves.replace("True", 'true')
        moves = moves.replace("False", 'false')
        moves = json.loads(moves)
        holds = np.zeros(self.board_size)
        for hold in moves:
            x, y = get_hold_coords(hold['Description'].upper())
            holds[x, y] = 1
        curr_grade_ind = french_grade_order.index(grade)
        if self.scale_min > curr_grade_ind :
            grade_num = 0
        elif self.scale_max < curr_grade_ind :
            grade_num = self.scale_max - self.scale_min
        else:
            grade_num = curr_grade_ind - self.scale_min

        # one_hot = np.zeros(self.scale_size)
        # one_hot[grade_num] = 1
        return holds.astype(np.float32), float(grade_num)
train_dataset = ClimbDataset("../modeldata/splits/train.csv", moonboard_2016_size, scale_bounds)
test_dataset = ClimbDataset("../modeldata/splits/test.csv", moonboard_2016_size, scale_bounds)
train_dataset.__getitem__(0)

(array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32),
 3.0)

In [257]:
import torch.nn as nn
import torch.nn.functional as F


device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

input_size = moonboard_2016_size[0] * moonboard_2016_size[1]
output_size = scale_size
class NeuralNetwork_Orig(nn.Module):
    def __init__(self):
        super(NeuralNetwork_Orig, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, 2048),
            nn.ReLU(),
            nn.Linear(2048, 2048),
            nn.ReLU(),
            nn.Linear(2048, 2048),
            nn.ReLU(),
            nn.Linear(2048, 2048),
            nn.ReLU(),
            nn.Linear(2048, 2048),
            nn.ReLU(),
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 1),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

Using cpu device


In [258]:
model = NeuralNetwork_Orig().to(device)
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

batch_size = 32

train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

In [259]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        y = y.float()
        pred = model(X)
        # print(pred)
        # print(y)
        
        loss = loss_fn(pred, y)
        # print(loss.dtype)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # if batch % 100 == 0:
        #     loss, current = loss.item(), batch * len(X)
        #     print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, error = 0, 0
    with torch.no_grad():  # Why do we do this?
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            y = y.float()
            pred = model(X)
            test_loss += loss_fn(pred, y)
            error += ((pred.squeeze() - y).abs()).sum().item()
    test_loss /= num_batches
    error /= size
    return (error, test_loss)

In [260]:
epochs = 400
train_accuracies = []
train_losses = []
val_accuracies = []
val_losses = []
best_accuracy = 0
for t in range(epochs):
    train(train_dataloader, model, loss_fn, optimizer)
    train_accuracy, train_loss = test(train_dataloader, model, loss_fn)
    print(train_accuracy)
    val_accuracy, val_loss =  test(test_dataloader, model, loss_fn)
    train_accuracies.append(train_accuracy)
    val_accuracies.append(val_accuracy)
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    if t % 50 == 0:
        print(train_accuracy)

    if t % 100 == 0:
        print(f"At Epoch {t} model has Accuracy: {(100*val_accuracy):>0.1f}%, Avg loss: {val_loss:>8f} \n")\
        
    if val_accuracy > best_accuracy:
        print(f"At Epoch {t} new best model found with Accuracy: {(100*val_accuracy):>0.1f}%, Avg loss: {val_loss:>8f} \n")
        best_accuracy = val_accuracy

print("Done!")

/Users/avery/Library/Python/3.9/lib/python/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/avery/Library/Python/3.9/lib/python/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([17])) that is different to the input size (torch.Size([17, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/avery/Library/Python/3.9/lib/python/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([13])) that is different to the input size (torch.Size([13, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure 

3.448611746208772
3.448611746208772
At Epoch 0 model has Accuracy: 390.5%, Avg loss: 24.316093 

At Epoch 0 new best model found with Accuracy: 390.5%, Avg loss: 24.316093 

3.438440459429805
3.428306456250902
3.4182232334762466
3.408185152754222
3.3982020283406107
3.38827683798986
3.378410127917283
3.3685978279378066
3.358821789607165
3.349076709350868
3.3393669745112677
3.3297056766360242
3.320093291461055
3.3105093623419006
3.3009480394887594
3.29140948148027
3.2819052997967937
3.272431565212047
3.2629835732264003
3.2535612533604574
3.244166931434261
3.234798264007921
3.2254542115103564
3.2161279902997797
3.206815036835351
3.1975142410665955
3.188230483691632
3.17896008216206
3.169697503845356
3.1604422377659045
3.151193200303005
3.1419546906997646
3.132725184839262
3.1235119244793546
3.114315138678077
3.105136818478344
3.095960295503189
3.0867876400848457
3.0776191341408956
3.0684546719522454
3.059295654296875
3.050150107015914
3.041002859419543
3.0318559025506775
3.022707985400052